<h4 style='color: cyan'>Recomendações de Filmes IMDB dataset </h4>

In [96]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import numpy as np

In [59]:
dados = pd.read_csv('Dados.csv')
tags = pd.read_csv('Tags.csv')
filme = pd.read_csv('Filmes.csv')
rating = pd.read_csv('Ratings.csv', sep=';')

In [60]:
dados.head(5)

,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription
0,Pulp Fiction,1994,8.9,94.0,154,"1,871,051",$107.93,\nCrime Drama,Quentin Tarantino John Travolta Uma Thurman Sa...,\nThe lives of two mob hitmen a boxer a gangst...
1,The Amazing Spider-Man 2,2014,6.6,53.0,142,"425,529",$202.85,\nAction Adventure Fantasy,Marc Webb Andrew Garfield Emma Stone Jamie Fox...,\nWhen New York is put under siege by Oscorp i...
2,The Shawshank Redemption,1994,9.3,80.0,142,"2,409,436",$28.34,\nDrama,Frank Darabont Tim Robbins Morgan Freeman Bob ...,\nTwo imprisoned men bond over a number of yea...
3,Star Wars: Episode IV - A New Hope,1977,8.6,90.0,121,"1,255,464",$322.74,\nAction Adventure Fantasy,George Lucas Mark Hamill Harrison Ford Carrie ...,\nLuke Skywalker joins forces with a Jedi Knig...
4,Back to the Future,1985,8.5,87.0,116,"1,087,878",$210.61,\nAdventure Comedy Sci-Fi,Robert Zemeckis Michael J. Fox Christopher Llo...,\nMarty McFly a 17-year-old high school studen...


In [61]:
dados.shape

(9937, 10)

In [62]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9937 entries, 0 to 9936
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Name            9937 non-null   object 
 1   Year            9754 non-null   object 
 2   Stars           9725 non-null   float64
 3   Score           6145 non-null   float64
 4   Time            9722 non-null   object 
 5   Votes           9736 non-null   object 
 6   Total           6892 non-null   object 
 7   Tags            9929 non-null   object 
 8   Directors_Cast  9853 non-null   object 
 9   Discription     6145 non-null   object 
dtypes: float64(2), object(8)
memory usage: 776.5+ KB


In [63]:
dados['Votes'] = dados['Votes'].str.replace(',','').astype(float)

In [64]:
dados['Total'] = dados['Total'].str.replace('$','').astype(float)

In [65]:
dados.head()

,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription
0,Pulp Fiction,1994,8.9,94.0,154,1871051.0,107.93,\nCrime Drama,Quentin Tarantino John Travolta Uma Thurman Sa...,\nThe lives of two mob hitmen a boxer a gangst...
1,The Amazing Spider-Man 2,2014,6.6,53.0,142,425529.0,202.85,\nAction Adventure Fantasy,Marc Webb Andrew Garfield Emma Stone Jamie Fox...,\nWhen New York is put under siege by Oscorp i...
2,The Shawshank Redemption,1994,9.3,80.0,142,2409436.0,28.34,\nDrama,Frank Darabont Tim Robbins Morgan Freeman Bob ...,\nTwo imprisoned men bond over a number of yea...
3,Star Wars: Episode IV - A New Hope,1977,8.6,90.0,121,1255464.0,322.74,\nAction Adventure Fantasy,George Lucas Mark Hamill Harrison Ford Carrie ...,\nLuke Skywalker joins forces with a Jedi Knig...
4,Back to the Future,1985,8.5,87.0,116,1087878.0,210.61,\nAdventure Comedy Sci-Fi,Robert Zemeckis Michael J. Fox Christopher Llo...,\nMarty McFly a 17-year-old high school studen...


In [66]:
dados['Tags'] = dados['Tags'].str.replace('\\n', '').str.split().str.join(',')

In [67]:
dados['Discription'] = dados['Discription'].str.replace('\\n', '')

In [68]:
tags.head()

,movieId,tag
0,1,"Owned,imdb top 250,Pixar,Pixar,time travel,chi..."
1,2,"Robin Williams,time travel,fantasy,based on ch..."
2,3,"funny,best friend,duringcreditsstinger,fishing..."
3,4,"based on novel or book,chick flick,divorce,int..."
4,5,"aging,baby,confidence,contraception,daughter,g..."


In [69]:
tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45256 entries, 0 to 45255
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  45256 non-null  object
 1   tag      45256 non-null  object
dtypes: object(2)
memory usage: 707.2+ KB


In [71]:
filme['movieId'] = filme['movieId'].apply(lambda x: str(x))

In [93]:
filme.head(1)

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995


In [94]:
dados.head(1)

,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription
0,Pulp Fiction,1994,8.9,94.0,154,1871051.0,107.93,"Crime,Drama",Quentin Tarantino John Travolta Uma Thurman Sa...,The lives of two mob hitmen a boxer a gangster...


In [95]:
tags.head(1)

,movieId,tag
0,1,"Owned,imdb top 250,Pixar,Pixar,time travel,chi..."


In [74]:
df2 = filme.merge(dados, left_on='title', right_on='Name')
df2 = df2.merge(tags, left_on='movieId', right_on='movieId')
df2['Infos'] = df2['genres'] + str(df2['Directors_Cast']) + str(df2['Discription']) + df2['tag']
df2.head(2)

,movieId,title,genres,year,Name,Year,Stars,Score,Time,Votes,Total,Tags,Directors_Cast,Discription,tag,Infos
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,Toy Story,1995,8.3,95.0,81,908794.0,191.80,"Animation,Adventure,Comedy",John Lasseter Tom Hanks Tim Allen Don Rickles ...,A cowboy doll is profoundly threatened and jea...,"Owned,imdb top 250,Pixar,Pixar,time travel,chi...",Adventure|Animation|Children|Comedy|Fantasy0 ...
1,2,Jumanji,Adventure|Children|Fantasy,1995,Jumanji,1995,7.0,39.0,104,316485.0,100.48,"Adventure,Comedy,Family",Joe Johnston Robin Williams Kirsten Dunst Bonn...,When two kids find and play a magical board ga...,"Robin Williams,time travel,fantasy,based on ch...",Adventure|Children|Fantasy0 John Lassete...


In [79]:
vetor = TfidfVectorizer()
tfidf = vetor.fit_transform(df2['Infos'].apply(lambda x: np.str_(x)))

In [83]:
similar_coss = cosine_similarity(tfidf)
similar_coss

array([[1.        , 0.0845192 , 0.01541288, ..., 0.00750217, 0.00791154,
        0.04709016],
       [0.0845192 , 1.        , 0.14576225, ..., 0.04183801, 0.12121005,
        0.24228852],
       [0.01541288, 0.14576225, 1.        , ..., 0.04537114, 0.03740296,
        0.27669818],
       ...,
       [0.00750217, 0.04183801, 0.04537114, ..., 1.        , 0.07189066,
        0.0468979 ],
       [0.00791154, 0.12121005, 0.03740296, ..., 0.07189066, 1.        ,
        0.04996566],
       [0.04709016, 0.24228852, 0.27669818, ..., 0.0468979 , 0.04996566,
        1.        ]])

In [84]:
df_cossi = pd.DataFrame(similar_coss, columns=df2['title'], index=df2['title'])
df_cossi.head()

title,Toy Story,Jumanji,Waiting to Exhale,Father of the Bride Part II,Heat,Heat,Sabrina,Sabrina,Tom and Huck,GoldenEye,...,The Nut Job 2: Nutty by Nature,Geostorm,The Shining,The Greatest Showman,Serving in Silence: The Margarethe Cammermeyer Story,Tomb Raider,A Wrinkle in Time,Pacific Rim: Uprising,Rampage,The Man Who Killed Don Quixote
title,,,,,,,,,,,,,,,,,,,,,
Toy Story,1.000000,0.084519,0.015413,0.033720,0.013419,0.013419,0.006743,0.017602,0.052154,0.018180,...,0.269396,0.009534,0.025451,0.003830,0.012982,0.013784,0.082331,0.007502,0.007912,0.047090
Jumanji,0.084519,1.000000,0.145762,0.098174,0.008898,0.008898,0.054817,0.022193,0.081527,0.014927,...,0.079557,0.018045,0.127883,0.013779,0.057568,0.221738,0.116536,0.041838,0.121210,0.242289
Waiting to Exhale,0.015413,0.145762,1.000000,0.240096,0.052270,0.052270,0.268647,0.113880,0.273885,0.028647,...,0.257490,0.050570,0.304581,0.030163,0.237071,0.044348,0.134722,0.045371,0.037403,0.276698
Father of the Bride Part II,0.033720,0.098174,0.240096,1.000000,0.036146,0.036146,0.078492,0.048594,0.213205,0.026780,...,0.314908,0.051978,0.276885,0.022475,0.215033,0.053564,0.115797,0.096973,0.027940,0.237150
Heat,0.013419,0.008898,0.052270,0.036146,1.000000,1.000000,0.036815,0.007383,0.037625,0.018608,...,0.042315,0.012687,0.049435,0.051098,0.038451,0.053161,0.026296,0.033482,0.042225,0.035256


In [85]:
def sistema_rec(texto):
    sistema_recomendacao = pd.DataFrame(df_cossi[f"{texto}"].sort_values(ascending=False))
    sistema_recomendacao.columns = ['Recomendações']
    return sistema_recomendacao.head(10)[1:].reset_index(names=['Titulo'])

In [86]:
sistema_rec('Thor')

,Titulo,Recomendações
0,Thor: The Dark World,0.724407
1,Thor: Ragnarok,0.599314
2,Captain America: The First Avenger,0.559985
3,Avengers: Age of Ultron,0.559308
4,Ant-Man,0.521324
5,Elektra,0.501664
6,Fantastic Four,0.478854
7,Fantastic Four,0.478854
8,Spider-Man 2,0.476222
